In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json

In [2]:
def getMovieDetails(imdbID):
    data = {}
    
    movie_url = "https://www.imdb.com/title/"+imdbID
    r = requests.get(url=movie_url)
    # Create a BeautifulSoup object
    soup = BeautifulSoup(r.text, 'html.parser')
    jsonData = soup.find('script',{"type":"application/ld+json"})
    #print(jsonData.string)
    Moredata=[]
    Moredata.append(json.loads(jsonData.string))
    data["expanded"]=Moredata

    

    #imdbId
    data["imdbID"] = imdbID

    #page title
    title = soup.find('title')
    data["title"] = title.string

    #title Year


    #RunTime
    data["RunTime"]=""
    data["Minutes"]=""
    runTime = soup.find("time")
    if runTime!= None :
      data["RunTime"]=runTime.string.strip()
      data["Minutes"]=runTime['datetime']





    # rating
    data["ratingValue"]=""
    ratingValue = soup.find("span", {"itemprop" : "ratingValue"})
    if ratingValue!= None :
        data["ratingValue"] = ratingValue.string

    # no of rating given
    data["ratingCount"] =""
    ratingCount = soup.find("span", {"itemprop" : "ratingCount"})
    if ratingCount!= None  :
        data["ratingCount"] = ratingCount.string

    # name
    data["name"]=""
    titleName = soup.find("div",{'class':'titleBar'}).find("h1")
    if titleName!= None :
        data["name"] = titleName.contents[0].replace(u'\xa0', u'')

    # additional details
    subtext = soup.find("div",{'class':'subtext'})
    data["subtext"] = ""
    for i in subtext.contents:
        data["subtext"] += i.string.strip()

    # summary
    summary_text = soup.find("div",{'class':'summary_text'})
   # print(summary_text)
    if summary_text!=None and summary_text.string != None :
      data["summary_text"] = summary_text.string.strip()
    else :
      data["summary_text"]=""
    credit_summary_item = soup.find_all("div",{'class':'credit_summary_item'})
    data["credits"] = {}
    for i in credit_summary_item:
        item = i.find("h4")
        names = i.find_all("a")
        data["credits"][item.string] = []
        for i in names:
            data["credits"][item.string].append({
                "link": i["href"],
                "name": i.string
            })

    return data

In [6]:
getMovieDetails('tt2631186')


{'Minutes': 'PT159M',
 'RunTime': '2h 39min',
 'credits': {'Director:': [{'link': '/name/nm1442514/',
    'name': 'S.S. Rajamouli'}],
  'Stars:': [{'link': '/name/nm1659141/', 'name': 'Prabhas'},
   {'link': '/name/nm2281292/', 'name': 'Rana Daggubati'},
   {'link': '/name/nm0471447/', 'name': 'Ramya Krishnan'},
   {'link': 'fullcredits/', 'name': 'See full cast & crew'}],
  'Writers:': [{'link': '/name/nm2353436/', 'name': 'Vijayendra Prasad'},
   {'link': '/name/nm1442514/', 'name': 'S.S. Rajamouli'},
   {'link': 'fullcredits#writers/', 'name': '4 more credits'}]},
 'expanded': [{'@context': 'http://schema.org',
   '@type': 'Movie',
   'actor': [{'@type': 'Person', 'name': 'Prabhas', 'url': '/name/nm1659141/'},
    {'@type': 'Person', 'name': 'Rana Daggubati', 'url': '/name/nm2281292/'},
    {'@type': 'Person', 'name': 'Ramya Krishnan', 'url': '/name/nm0471447/'},
    {'@type': 'Person', 'name': 'Sathyaraj', 'url': '/name/nm0766470/'}],
   'aggregateRating': {'@type': 'AggregateRatin

In [4]:

def getCrewData(imdbID):

    url = "https://www.imdb.com/title/"+imdbID+"/fullcredits/"
    crew_data = {
        "imdbID" : imdbID,
        "crew": []
    }
    r = requests.get(url=url)

    # Create a BeautifulSoup object
    soup = BeautifulSoup(r.text, 'html.parser')

    #page title
    title = soup.find('title')
    crew_data["title"]=""
    if title!=None :
        crew_data["title"] = title.string
    cast_list = soup.find("table", {"class" : "cast_list"})
    trows=[]
    if cast_list!=None :
        trows = cast_list.find_all('tr')

    for tr in trows:
        td = tr.find_all('td')
        if len(td)==4:
            row = [i.text for i in td]
            crew_data["crew"].append({
                "name":re.sub("[^a-zA-Z' ]+", '', row[1]).strip(),
                "character":re.sub("[^a-zA-Z' ]+", '', row[3]).strip()
            })
    return crew_data

In [7]:
getCrewData('tt2631186')


{'crew': [{'character': 'Shivudu               Amarendra Baahubali               Mahendra Baahubali',
   'name': 'Prabhas'},
  {'character': 'Bhallaladeva', 'name': 'Rana Daggubati'},
  {'character': 'Shivagami', 'name': 'Ramya Krishnan'},
  {'character': 'Kattappa', 'name': 'Sathyaraj'},
  {'character': 'Devasena', 'name': 'Anushka Shetty'},
  {'character': 'Deependra Singh', 'name': 'Deependra Singh'},
  {'character': 'Avanthika', 'name': 'Tamannaah Bhatia'},
  {'character': 'Bijjaladeva', 'name': 'Nassar'},
  {'character': 'Aslaam Khan', 'name': 'Sudeep'},
  {'character': 'Kalakeya War Lord Inkoshi', 'name': 'Prabhakar'},
  {'character': 'Bhalladeva Samantha Raju', 'name': 'Rajsekhar Aningi'},
  {'character': 'Swamiji', 'name': 'Tanikella Bharani'},
  {'character': 'Kalki Brothers', 'name': 'John Kokken'},
  {'character': 'Sanga', 'name': 'Rohini'},
  {'character': 'Bhadra', 'name': 'Adivi Sesh'},
  {'character': 'Hero friend', 'name': 'Venkat Vadisetti'},
  {'character': 'Tapsi Awh

In [9]:

def getImages(ImdbId) :
  url = "https://www.imdb.com/title/"+ImdbId+"/mediaindex"
  data= {}
  data['ImdbId']=ImdbId
  image_urls = []
  r = requests.get(url=url)
  soup = BeautifulSoup(r.text, 'html.parser')
  img =soup.find("img",{'class':'poster'})
  src=""
  if img!=None :
      src = img.get('src')
  data["poster_url"]= src
  
  imagelist = soup.find('div',{"class" : "media_index_thumb_list"})
  
  Image_anchors_list=[]
  if imagelist!= None :
      Image_anchors_list=imagelist.findAll('a')

  
  for a in Image_anchors_list :
    img = { }
    img["image_title"]=""
    if a.has_attr('title'):
        img["image_title"] = a['title']
    img['url']=""
    #print(a['title'])
    imageTagList = a.findAll('img')
    if len(imageTagList) > 0 :
      img['url'] = imageTagList[0]['src']
    image_urls.append(img)

  data['other_images'] = image_urls 

    
  return data



In [11]:
getImages('tt2631186')

{'ImdbId': 'tt2631186',
 'other_images': [{'image_title': 'Senthil Kumar in Baahubali: The Beginning (2015)',
   'url': 'https://m.media-amazon.com/images/M/MV5BNjk2ZDRmZjQtMjYwZi00YzIzLThiNmQtODcxNGJhNTE5MDhkXkEyXkFqcGdeQXVyNjY1MTg4Mzc@._V1_UY100_CR68,0,100,100_AL_.jpg'},
  {'image_title': 'Baahubali: The Beginning (2015)',
   'url': 'https://m.media-amazon.com/images/M/MV5BNTI5Y2Y1YmEtMTAwZi00ZjU0LTg5NzEtMDBiYjk5Njc2ZGM4XkEyXkFqcGdeQXVyNjY1MTg4Mzc@._V1_UY100_CR68,0,100,100_AL_.jpg'},
  {'image_title': 'Prabhas, Scarlett Mellish Wilson, Nora Fatehi, and Madhussneha Upadhyay in Baahubali: The Beginning (2015)',
   'url': 'https://m.media-amazon.com/images/M/MV5BNWFlMjVlZjMtNDdlYi00ODAyLThiMjQtMTE5YmVjNWE4NTcxXkEyXkFqcGdeQXVyNjY1MTg4Mzc@._V1_UY100_CR68,0,100,100_AL_.jpg'},
  {'image_title': 'Baahubali: The Beginning (2015)',
   'url': 'https://m.media-amazon.com/images/M/MV5BZTY0NDg2OWQtNTUzOC00NWRlLWJiNjQtN2E1Njg5NTQ4ZTFjXkEyXkFqcGdeQXVyNjY1MTg4Mzc@._V1_UY100_CR68,0,100,100_AL_.jpg'},


In [12]:
def getVideos(ImdbId) :
  url = "https://www.imdb.com/title/"+ImdbId+"/mediaindex"
  data= {}
  data['ImdbId']=ImdbId
  video_urls = []
  r = requests.get(url=url)
#  print(url)
  soup = BeautifulSoup(r.text, 'html.parser')
  videoList =soup.find("div",{'class':'mediastrip_big'})
#  print(videoList)
  VideoAnchorList=[]
  if videoList != None :
      VideoAnchorList = videoList.findAll('a')
  for a in VideoAnchorList :
#    print(a)
    video_urls.append(a['href'])
  data['video_urls']=video_urls
#  print(data)
  return data



In [13]:
getVideos('tt2631186')


{'ImdbId': 'tt2631186',
 'video_urls': ['/title/tt2631186/videoplayer/vi3834163993',
  '/title/tt2631186/videoplayer/vi657702425',
  '/title/tt2631186/videoplayer/vi1228193305']}

**Scrape all details**

In [14]:
def scrapIMDB(ImdbId) :
    data = {}
    data['imdbId']=ImdbId
    data['videos'] = getVideos(ImdbId)
    data['images'] = getImages(ImdbId)
    data['info']    = getMovieDetails(ImdbId)
    data['crew_data'] =  getCrewData(ImdbId)
    
    return data    
    

In [15]:

listId = ["tt5074352","tt6980546","tt7838158","tt7465992","tt2631186","tt7392212"]


**Start scraping multiple movies and download each file**

In [16]:
count=0
for id in listId :
  print(id)  
  data=scrapIMDB(id)
  with open(id +'.json', 'w') as json_file:
      json.dump(data, json_file)
  print(count)
  count=count+1    


tt5074352
0
tt6980546
1
tt7838158
2
tt7465992
3
tt2631186
4
tt7392212
5
